In [1]:
import numpy as np

#Input values
fck = 55
fyk = 500

t = 550 
a = 70

nx, ny, nxy = -250, -200, 750
mx, my, mxy = 20, -100, 20
vx, vy = 100, 300

c = 200
ø = 20

Asx1 = np.pi * ø**2 / 4 * 1000/c
Asx2 = np.pi * ø**2 / 4 * 1000/c 
Asy1 = np.pi * ø**2 / 4 * 1000/c
Asy2 = np.pi * ø**2 / 4 * 1000/c

# Design strenghts
fcd = 0.85 * fck / 1.5
fyd = fyk / 1.15

# Define cracked and uncracked thicknesses
t_uncracked = t / 2
t_cracked = 2 * a

# Initial layer thicknesses - assume both layers are uncracked
t1 = t_uncracked
t2 = t_uncracked

# Function to calculate forces based on k1, k2, z
def calculate_forces(k1, k2, z):
    nx1 = k1 * nx + mx / (z * 1e-3)
    nx2 = k2 * nx - mx / (z * 1e-3)
    ny1 = k1 * ny + my / (z * 1e-3)
    ny2 = k2 * ny - my / (z * 1e-3)
    nxy1 = k1 * nxy + mxy / (z * 1e-3)
    nxy2 = k2 * nxy - mxy / (z * 1e-3)
    return nx1, nx2, ny1, ny2, nxy1, nxy2

# Function to compute max and min normal force in a layer
def calculate_n_max(nx, ny, nxy):
    return (nx + ny) / 2 + np.sqrt(((nx - ny) / 2) ** 2 + nxy ** 2)

def calculate_n_min(nx, ny, nxy):
    return (nx + ny) / 2 - np.sqrt(((nx - ny) / 2) ** 2 + nxy ** 2)

# **Iterate through the layers to check for cracks and update values**
for layer in [1, 2]:
    k1 = (t - t2) / (2 * t - t1 - t2)
    k2 = 1 - k1
    z = t - 1 / 2 * (t1 + t2)

    nx1, nx2, ny1, ny2, nxy1, nxy2 = calculate_forces(k1, k2, z)

    n_max = calculate_n_max(nx1, ny1, nxy1) if layer == 1 else calculate_n_max(nx2, ny2, nxy2)

    if n_max > 0:  # If the layer is cracked
        print(f"Layer {layer} is cracked")

        if layer == 1:
            t1 = t_cracked
        else:
            t2 = t_cracked

        k1 = (t - t2) / (2 * t - t1 - t2)
        k2 = 1 - k1
        z = t - 1 / 2 * (t1 + t2)

        nx1, nx2, ny1, ny2, nxy1, nxy2 = calculate_forces(k1, k2, z)

    else:
        print(f"Layer {layer} is uncracked")

# **Final Forces Matrices**
n1 = np.array([round(nx1), round(ny1), round(nxy1)])
n2 = np.array([round(nx2), round(ny2), round(nxy2)])

# **Final n_max values**
n_max1 = round(calculate_n_max(nx1, ny1, nxy1))
n_max2 = round(calculate_n_max(nx2, ny2, nxy2))

print("\nFinal n_max1:", n_max1, "kN/m")
print("Final n_max2:", n_max2, "kN/m")
print("\nFinal forces for Layer 1 (n1):", n1, "kN/m")
print("Final forces for Layer 2 (n2):", n2, "kN/m")

sigma_rdmax = 0.6 * (1 - fck / 250) * fcd

# **Function to solve rho quadratic equation with different Asx for Layer 1 and 2**
def solve_rho(nx, ny, nxy, Asx, Asy):
    if nxy == 0:
        return None  

    a = 1
    b = (nx / nxy) - (ny / nxy) * (Asx / Asy)
    c = -Asx / Asy
    roots = np.roots([a, b, c])

    # Convert to angles and filter valid range
    rho_degrees = np.degrees(np.arctan(roots))
    valid_rho = [r for r in rho_degrees if -90 <= r <= 90]

    if valid_rho:
        return np.radians(min(valid_rho))  # Choose the most negative valid angle
    else:
        return None  

# **Check for cracked vs uncracked layers**
sigma_c1 = sigma_c2 = float('-inf')  
fx1 = fy1 = fx2 = fy2 = None  

for layer, (nx, ny, nxy) in zip([1, 2], [n1, n2]):
    Asx = Asx1 if layer == 1 else Asx2  # Different Asx for each layer
    Asy = Asy1 if layer == 1 else Asy2

    if calculate_n_max(nx, ny, nxy) > 0:
        rho_rad = solve_rho(nx, ny, nxy, Asx, Asy)
        if rho_rad is None:
            print(f"\nNo valid crack angle found for Layer {layer}")
            continue  

        # Compute forces and stresses
        fx = nx + nxy * np.tan(rho_rad)
        fy = ny + nxy / np.tan(rho_rad)
        fc = abs(nxy / (np.sin(rho_rad) * np.cos(rho_rad)))
        sigma_c = fc / (t1 if layer == 1 else t2)

        if layer == 1:
            sigma_c1 = sigma_c
            fx1, fy1 = fx, fy
        else:
            sigma_c2 = sigma_c
            fx2, fy2 = fx, fy

        print(f"\nLayer {layer} (Cracked):")
        print(f"Crack angle (𝜙) = {np.degrees(rho_rad):.2f}°")  
        print(f"Force in x-direction (f_sx) = {fx:.1f} kN/m")
        print(f"Force in y-direction (f_sy) = {fy:.1f} kN/m")
        print(f"Concrete force (f_c) = {fc:.1f} kN/m")
        print(f"Concrete stress (σ_c{layer}) = {sigma_c:.2f} MPa")

    else:
        n_min = calculate_n_min(nx, ny, nxy)
        sigma_c = n_min / (t1 if layer == 1 else t2)

        if layer == 1:
            sigma_c1 = sigma_c
        else:
            sigma_c2 = sigma_c

        print(f"\nLayer {layer} (Uncracked): Concrete stress (σ_c{layer}) = {sigma_c:.2f} MPa")

print(f"\nMaximum allowed concrete stress: σ_Rd,max = {round(sigma_rdmax, 1)} MPa")

# **Compute Necessary Reinforcement Areas**
Asx1_neccessary = np.ceil(fx1 * 1e3 / fyd) if fx1 else 0
Asy1_neccessary = np.ceil(fy1 * 1e3 / fyd) if fy1 else 0
Asx2_neccessary = np.ceil(fx2 * 1e3 / fyd) if fx2 else 0
Asy2_neccessary = np.ceil(fy2 * 1e3 / fyd) if fy2 else 0

print("\nNecessary Reinforcement Areas:")
print(f"Asx1 = {Asx1_neccessary:.0f} mm²/m, Asy1 = {Asy1_neccessary:.0f} mm²/m")
print(f"Asx2 = {Asx2_neccessary:.0f} mm²/m, Asy2 = {Asy2_neccessary:.0f} mm²/m")


# Compute shear force magnitude
v0 = np.sqrt(vx**2 + vy**2)
phi0 = np.arctan(vy / vx)  # [Radians]

# Compute size effect factor (k)
k_factor = 1 + np.sqrt(200 / z)

# Compute reinforcement density (ρ₀)
rho0 = np.pi * ø**2 / (4*z*c)  

# Compute shear capacity without reinforcement (vRd,c)
Crd_c = 0.18 / 1.5  # Safety factor included
vRd_c = Crd_c * k_factor * (100 * rho0 * fck) ** (1 / 3) * z  

# Compute maximum shear capacity (vRd,max)
cot_theta = 2
v = 0.6 * (1 - fck / 250)  # Strength reduction factor
vRd_max = v * fcd / (cot_theta + np.tan(np.arctan(1 / cot_theta))) * z  

# **Store Original Forces Before Shear Reinforcement**
original_n1 = n1
original_n2 = n2

print(f"\nShear force magnitude (v0) = {round(v0)} kN/m")

print(f"\nShear Capacity without reinforcement (vRd_c) = {round(vRd_c)} kN/m")
print(f"Maximum shear capacity (vRd_max) = {round(vRd_max)} kN/m")


# **Check if additional shear reinforcement is required**
if v0 > vRd_c:
    print("\nShear reinforcement REQUIRED!")

    # Compute required shear reinforcement amount
    Asw_per_s = v0 * 1e6 / (z * fyd * cot_theta)  # [mm²/m²]

    # Compute additional membrane forces due to shear reinforcement
    delta_nx = 0.5 * (vx**2 / v0) * cot_theta  # [kN/m]
    delta_ny = 0.5 * (vy**2 / v0) * cot_theta  # [kN/m]
    delta_nxy = 0.5 * (vx * vy / v0) * cot_theta  # [kN/m]

    delta = np.array([delta_nx, delta_ny, delta_nxy])

    # **Ensure forces are updated correctly without accumulation**
    updated_n1 = original_n1 + delta
    updated_n2 = original_n2 + delta

    print("\nUpdated membrane forces after shear reinforcement:")
    print(f"Updated n1: [{round(updated_n1[0])}, {round(updated_n1[1])}, {round(updated_n1[2])}]")
    print(f"Updated n2: [{round(updated_n2[0])}, {round(updated_n2[1])}, {round(updated_n2[2])}]")
else:
    print("\nNo additional shear reinforcement required.")

# **Final Shear Check**
if v0 > vRd_max:
    print("\nThe section is OVERLOADED in shear (v0 > vRd,max)!")
else:
    print("\nThe section is SAFE for shear.")




Layer 1 is cracked
Layer 2 is cracked

Final n_max1: 234 kN/m
Final n_max2: 348 kN/m

Final forces for Layer 1 (n1): [ -76 -344  424] kN/m
Final forces for Layer 2 (n2): [-174  144  326] kN/m

Layer 1 (Cracked):
Crack angle (𝜙) = -53.77°
Force in x-direction (f_sx) = -654.7 kN/m
Force in y-direction (f_sy) = -654.7 kN/m
Concrete force (f_c) = 889.3 kN/m
Concrete stress (σ_c1) = 6.35 MPa

Layer 2 (Cracked):
Crack angle (𝜙) = -32.00°
Force in x-direction (f_sx) = -377.7 kN/m
Force in y-direction (f_sy) = -377.7 kN/m
Concrete force (f_c) = 725.4 kN/m
Concrete stress (σ_c2) = 5.18 MPa

Maximum allowed concrete stress: σ_Rd,max = 14.6 MPa

Necessary Reinforcement Areas:
Asx1 = -1505 mm²/m, Asy1 = -1505 mm²/m
Asx2 = -868 mm²/m, Asy2 = -868 mm²/m

Shear force magnitude (v0) = 316 kN/m

Shear Capacity without reinforcement (vRd_c) = 231 kN/m
Maximum shear capacity (vRd_max) = 2392 kN/m

Shear reinforcement REQUIRED!

Updated membrane forces after shear reinforcement:
Updated n1: [-44, -59, 519